# City Net #

*Warning - this script is slow to run - for each 10 minute file, it will take 20 seconds*

## Instructions ##
 
1. The only part of this script you will need to change is the directory of audio files and results folders.
    * Change these file paths to the paths of your input folder and output folder within the Bucket.
2. Once you have set the file directories, select Run -> Run all cells from the menu above.
3. The final argument should shut down the instance when this has finished, however if there is an error at some point or the script becomes unresponsive, then this argument will not be executed, so you should check in on progress of script periodically.
4. If scripts becomes unresponsive before it has completed, then restart the kernel. The script is unresponsive if the boxes next to the current command changes from [*] / [number] to [ ] 


**Source of potential errors:** 
* If you have multiple kernels running in this instance, then GPU allocation may be disrupted and it will throw errors - make sure there is only one notebook open when running this script. You can view kernels on the left, with the Stop Icon that is below the folder browser. Shut down all the kernels except for this script.

* The notebook may become unresponsive after running for a long time - you will have to restart notebook and code again. It will resume classifying from where it left off.

* Some files will not be analysed as there will be several 0MB files recorded by the AudioMoth, so don't be concerned if some files can't be analysed.



## Mount buckets ##

In [ ]:
# Mount raw data bucket - this bucket contains all the wav files 
!mountpoint -q /home/jupyter/gcs_raw && echo "mounted" || mkdir -p gcs_raw; gcsfuse --implicit-dirs --rename-dir-limit=100 --disable-http2 --max-conns-per-host=100 "acoustic-data-raw" "/home/jupyter/gcs_raw"
# Mount outputs bucket - results csvs will be written to this bucket
!mountpoint -q /home/jupyter/gcs_outputs && echo "mounted" || mkdir -p gcs_outputs; gcsfuse --implicit-dirs --rename-dir-limit=100 --disable-http2 --max-conns-per-host=100 "acoustic-processing-outputs" "/home/jupyter/gcs_outputs"

2022/12/02 12:01:33.821242 Start gcsfuse/0.41.8 (Go version go1.18.4) for app "" using mount point: /home/jupyter/gcs_raw
2022/12/02 12:01:33.839394 Opening GCS connection...
2022/12/02 12:01:33.980533 Mounting file system "acoustic-data-raw"...
2022/12/02 12:01:33.999700 File system has been successfully mounted.
2022/12/02 12:01:34.133526 Start gcsfuse/0.41.8 (Go version go1.18.4) for app "" using mount point: /home/jupyter/gcs_outputs
2022/12/02 12:01:34.147317 Opening GCS connection...
2022/12/02 12:01:34.250189 Mounting file system "acoustic-processing-outputs"...
2022/12/02 12:01:34.270755 File system has been successfully mounted.


# Set folders for input (i.e. sound folders to analyse) and output #

These folders will be mounted on the left, so will start with '/home/jupyter/' but they are accessing files from the Google Cloud Bucket that you mounted. Make sure input_folder is the raw audio folder, and the results_folder is within gcs_outputs.

**All you need to change from below is 'trial_data_2021' on both lines to the new folder.**

# &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; #

In [ ]:
folder = "/home/jupyter/gcs_raw/nr-acoustic-data/city-config/"
results = "/home/jupyter/gcs_outputs/nr-acoustic-data/citynet/"

# &uarr; &uarr; &uarr; &uarr; &uarr; &uarr; &uarr; &uarr; &uarr; &uarr; &uarr; &uarr; #

### Make sure libraries and scripts are loaded ###

In [ ]:
![ -d "/home/jupyter/CityNet" ] && echo "Scripts are downloaded" || git clone https://github.com/mdfirman/CityNet.git
!wget -O CityNet/multi_predict_.py "https://www.dropbox.com/s/wgg8zi118uqgy5e/multi_predict_.py?dl=0"

Scripts are downloaded


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
!pip show tensorflow && echo "tensorflow installed" || pip install tensorflow
!pip show librosa && echo "librosa installed" || pip install librosa
!pip show tf_slim && echo "tf_slim installed" || pip install tf_slim
!pip show PyYAML && echo "PyYAML installed" || pip install -U PyYAML

In [ ]:
cd /home/jupyter/CityNet
![ -d "/home/jupyter/CityNet/__MACOSX" ] && echo "Models are downloaded" || python demo.py

## Run classifier ##

In [ ]:
%run -i multi_predict_.py "$folder" "$results"

## Concatenate results ##

This step is quite slow, as there will be many csvs to concatenate.

In [ ]:
import glob as glob
import pandas as pd
import os
import datetime
from datetime import datetime, date

In [ ]:
results_list_anth = glob.glob(results +  '*/*/*anthrop.csv')
results_list_bio = glob.glob(results +  '*/*/*biotic.csv')

d = date.today()
d = d.strftime('%y%m%d')

In [ ]:
li = []

for filename in results_list_anth:
    df = pd.read_csv(filename)
    df = df.rename(columns={"Average sound": "Average anthropogenic sound"})
    li.append(df)

frame_anth = pd.concat(li, axis=0, ignore_index=True)


In [ ]:
li = []

for filename in results_list_bio:
    df = pd.read_csv(filename)
    df = df.rename(columns={"Average sound": "Average biotic sound"})
    li.append(df)

frame_bio = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame_merged =  pd.merge(frame_anth, frame_bio, how='inner', on = 'Filename')
frame_merged['SD'] = [os.path.basename(os.path.dirname(f)) for f in frame_merged['Filename']]
frame_merged['basename'] = [os.path.basename(f).split('.')[0] for f in frame_merged['Filename']]
frame_merged['files_timestamp'] = [datetime.strptime(f, '%Y%m%d_%H%M%S') for f in frame_merged['basename']]
frame_merged['hour'] = [f.hour for f in frame_merged['files_timestamp']]


frame_merged.columns = frame_merged.columns.str.replace(' ','_')
frame_merged.to_csv(results + d +  '_concatenated_results_city_net.csv')

In [ ]:
# Shutdown the notebook   
!sudo shutdown -h now 